<a href="https://colab.research.google.com/github/rkuo2000/GenAI/blob/main/Colab/pyngrok_LLM_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM HTTP Server
## using pyNgrok for TCP tunneling

In [1]:
!pip install flask
!pip install pyngrok
!pip install accelerate
!pip install einops

## LLM model

In [2]:
import transformers
from transformers import AutoModelForCausalLM , AutoTokenizer
import torch
torch.set_default_device("cuda")

model_name = "Q-bert/Mamba-130M" # torch_dtype = "auto"
#model_name = "Q-bert/Mamba-370M"
#model_name = "Q-bert/Mamba-790M"
#model_name = "Q-bert/Mamba-1B"
#model_name = "Q-bert/Mamba-3B"
#model_name = "Q-bert/Mamba-3B-slimpj"
#model_name = "ckip-joint/bloom-3b-zh"
#model_name = "Qwen/Qwen1.5-7B-Chat"
#model_name = "lmsys/vicuna-7b-v1.5-16k"
#model_name = "yentinglin/Taiwan-LLM-7B-v2.0.1-chat"
#model_name = "mistralai/Mistral-7B-Instruct-v0.2"
#model_name = "MediaTek-Research/Breeze-7B-Instruct-v0.1" # torch_dtype=torch.bfloat16

LLM = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype="auto", device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## HTTP Server with Ngrok

In [3]:
import getpass
import os
import threading

from flask import Flask, request, jsonify
import json

from pyngrok import ngrok, conf

## set ngrok authtoken
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


In [4]:
app = Flask(__name__)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL

 * ngrok tunnel "https://3780-34-126-65-43.ngrok-free.app" -> "http://127.0.0.1:5000/"


In [5]:

# Define Flask routes
@app.route("/")
def home():
    return "Hello from Colab!"

@app.route("/text", methods=['POST'])
def text():
    if request.method == 'POST':
        prompt = request.json['text']

        input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        output = LLM.generate(input_ids, max_length=64, num_beams=5, no_repeat_ngram_size=2)
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        return jsonify(generated_text)

# Start the Flask server in a new thread
#threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Mar/2024 15:02:04] "POST /text HTTP/1.1" 200 -
